<h1 style="color:green; font-size:30px; font-weight:700; text-align:center">
    <u><b>Advanced Transformation</b></u>  
</h1> 

<p></p> 
<strong> Author:</strong> Gurtej Bains <br> 
<strong> Data Source:</strong>   <br> 
<strong> Last Updated: </strong>  <br>  
<strong> Problem Statement:</strong> <br>

## Import Libraries

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

## Import data

In [2]:
hero_info = pd.read_csv("superhero_info - superhero_info.csv")
hero_info.columns = map(str.lower, hero_info.columns)
hero_info.head()

,hero|publisher,gender,race,alignment,hair color,eye color,skin color,measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [3]:
hero_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   hero|publisher  463 non-null    object
 1   gender          463 non-null    object
 2   race            463 non-null    object
 3   alignment       463 non-null    object
 4   hair color      463 non-null    object
 5   eye color       463 non-null    object
 6   skin color      463 non-null    object
 7   measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [4]:
hero_powers = pd.read_csv("superhero_powers - superhero_powers.csv")
hero_powers.columns = map(str.lower, hero_powers.columns)
hero_powers.head()

,hero_names,powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [5]:
hero_powers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


## Clean the data 

### Hero column 

In [6]:
## adding expand=True
hero_info[['hero', 'publisher']] = hero_info['hero|publisher'].str.split('|',expand=True)
hero_info = hero_info.drop(columns=['hero|publisher'])
hero_info.head(2)

,gender,race,alignment,hair color,eye color,skin color,measurements,hero,publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


### Height and Weight column 

In [7]:
## use .str.replace to replace all single quotes
hero_info['measurements']=hero_info['measurements'].str.replace("'",'"')

## Apply the json.loads to the full column
hero_info['measurements']=hero_info['measurements'].apply(json.loads)

## New columns 
hero_info[['height', 'weight']]=hero_info['measurements'].apply(pd.Series)

## Numeric part only 
hero_info['height'] = hero_info['height'].str.extract('(\d+)')
hero_info['weight'] = hero_info['weight'].str.extract('(\d+)')

## Drop previous column 
hero_info = hero_info.drop(columns=['measurements'])

## qa 
hero_info

,gender,race,alignment,hair color,eye color,skin color,hero,publisher,height,weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203,441
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191,65
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185,90
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203,441
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193,122
...,...,...,...,...,...,...,...,...,...,...
458,Male,Human,good,Blond,blue,Unknown,Yellowjacket,Marvel Comics,183,83
459,Female,Human,good,Strawberry Blond,blue,Unknown,Yellowjacket II,Marvel Comics,165,52
460,Male,Yoda's species,good,White,brown,green,Yoda,George Lucas,66,17
461,Female,Human,good,Black,blue,Unknown,Zatanna,DC Comics,170,57


### Super powers column OHE

In [8]:
## showing the lists are really strings
hero_powers.loc[0,'powers']

'Agility,Super Strength,Stamina,Super Speed'

In [9]:
new_powers = hero_powers["powers"].str.split(",")
new_powers

0        [Agility, Super Strength, Stamina, Super Speed]
1      [Accelerated Healing, Durability, Longevity, S...
2      [Agility, Accelerated Healing, Cold Resistance...
3                                   [Lantern Power Ring]
4      [Accelerated Healing, Intelligence, Super Stre...
                             ...                        
662               [Flight, Energy Blasts, Size Changing]
663    [Cold Resistance, Durability, Longevity, Super...
664    [Agility, Stealth, Danger Sense, Marksmanship,...
665    [Cryokinesis, Telepathy, Magic, Fire Control, ...
666    [Super Speed, Intangibility, Time Travel, Time...
Name: powers, Length: 667, dtype: object

In [10]:
cols_to_make = new_powers.explode("powers").dropna().sort_values().unique()
cols_to_make

array(['Accelerated Healing', 'Adaptation', 'Agility',
       'Animal Attributes', 'Animal Control', 'Animal Oriented Powers',
       'Animation', 'Anti-Gravity', 'Astral Projection', 'Astral Travel',
       'Audio Control', 'Banish', 'Biokinesis', 'Camouflage',
       'Changing Armor', 'Clairvoyance', 'Cloaking', 'Cold Resistance',
       'Cryokinesis', 'Danger Sense', 'Darkforce Manipulation',
       'Death Touch', 'Density Control', 'Dexterity',
       'Dimensional Awareness', 'Dimensional Travel', 'Duplication',
       'Durability', 'Echolocation', 'Elasticity', 'Electrical Transport',
       'Electrokinesis', 'Element Control',
       'Elemental Transmogrification', 'Empathy', 'Energy Absorption',
       'Energy Armor', 'Energy Beams', 'Energy Blasts',
       'Energy Constructs', 'Energy Manipulation', 'Energy Resistance',
       'Enhanced Hearing', 'Enhanced Memory', 'Enhanced Senses',
       'Enhanced Sight', 'Enhanced Smell', 'Enhanced Touch',
       'Fire Control', 'Fire Resis

In [11]:
import warnings
warnings.filterwarnings("ignore")

for col in cols_to_make:
    hero_powers[col] = hero_powers['powers'].str.contains(col)

In [12]:
hero_powers.head(2)

,hero_names,powers,Accelerated Healing,Adaptation,Agility,Animal Attributes,Animal Control,Animal Oriented Powers,Animation,Anti-Gravity,...,Vision - Thermal,Vision - X-Ray,Vitakinesis,Wallcrawling,Water Control,Weapon-based Powers,Weapons Master,Weather Control,Web Creation,Wind Control
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
hero_powers = hero_powers.drop(columns=['powers'])

## Merge dataframes

In [16]:
hero_info.shape

(463, 10)

In [17]:
hero_powers.shape

(667, 168)

In [44]:
superhero_final = pd.merge(hero_info, hero_powers, how='inner', left_on='hero', right_on='hero_names')
superhero_final = superhero_final.drop(columns='hero_names')
superhero_final.columns = map(str.lower, superhero_final.columns)
superhero_final.columns = superhero_final.columns.str.strip()
superhero_final.columns = superhero_final.columns.str.replace('-', '')
superhero_final.columns = superhero_final.columns.str.replace(' ', '_')

superhero_final

,gender,race,alignment,hair_color,eye_color,skin_color,hero,publisher,height,weight,...,vision__thermal,vision__xray,vitakinesis,wallcrawling,water_control,weaponbased_powers,weapons_master,weather_control,web_creation,wind_control
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203,441,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191,65,...,False,False,False,False,False,False,True,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185,90,...,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203,441,...,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193,122,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Male,Human,good,Blond,blue,Unknown,Yellowjacket,Marvel Comics,183,83,...,False,False,False,False,False,False,False,False,False,False
459,Female,Human,good,Strawberry Blond,blue,Unknown,Yellowjacket II,Marvel Comics,165,52,...,False,False,False,False,False,False,False,False,False,False
460,Male,Yoda's species,good,White,brown,green,Yoda,George Lucas,66,17,...,False,False,False,False,False,False,True,False,False,False
461,Female,Human,good,Black,blue,Unknown,Zatanna,DC Comics,170,57,...,False,False,False,False,True,False,False,True,False,False


In [22]:
superhero_final.shape

(463, 177)

## Basic Analysis 

### average weight

In [48]:
q = """
SELECT super_speed, avg(weight) as weight
from superhero_final
group by 1 


"""
pysqldf(q)

,super_speed,weight
0,0,101.773585
1,1,129.404040


### average height 

In [50]:
q = """
SELECT publisher, avg(height) as height
from superhero_final
group by 1 
order by 2 desc 

"""
pysqldf(q)

,publisher,height
0,Image Comics,211.000000
1,Marvel Comics,191.545455
2,DC Comics,181.920290
3,Star Trek,181.500000
4,Team Epic TV,180.750000
5,Unknown,178.000000
6,Dark Horse Comics,176.909091
7,Shueisha,171.500000
8,George Lucas,159.600000
